## Model Training

In [1]:
import os

In [2]:
%pwd

'/home/utpal108/dev/Python/PW_Skills_Projects/plant-disease-detection/notebooks'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/home/utpal108/dev/Python/PW_Skills_Projects/plant-disease-detection'

In [5]:
# Config Entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [6]:
from plantDiseaseDetection.constants import *
from plantDiseaseDetection.utils import create_directories, read_yaml

In [7]:
# Configuration Manager
class ConfigurationManager:
    def __init__(self, config_filepath= CONFIG_FILE_PATH, params_filepath= PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    
    def get_training_config(self) -> TrainingConfig:
        base_model_config = self.config.prepare_base_model
        training_config = self.config.training
        data_ingestion_config = self.config.data_ingestion
        params = self.params

        create_directories([Path(training_config.root_dir)])
        
        training_config = TrainingConfig(
            root_dir = Path(training_config.root_dir),
            trained_model_path = Path(training_config.trained_model_path),
            updated_base_model_path = Path(base_model_config.updated_base_model_path),
            training_data = Path(data_ingestion_config.dataset_dir),
            params_epochs =  params.EPOCHS,
            params_batch_size = params.BATCH_SIZE,
            params_is_augmentation = params.AUGMENTATION,
            params_image_size = params.IMAGE_SIZE
        )

        return training_config

In [8]:
import tensorflow as tf

2023-11-14 09:06:33.693987: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-14 09:06:33.732664: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-14 09:06:33.732700: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-14 09:06:33.732736: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-14 09:06:33.740175: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-14 09:06:33.740709: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [9]:
class Traing:
    def __init__(self, config:TrainingConfig):
        self.config = config
    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(self.config.updated_base_model_path)

    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.20
        )

        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = 'bilinear'
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory = self.config.training_data,
            subset = 'validation',
            shuffle = False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range = 45,
                horizontal_flip = True,
                width_shift_range = 0.2,
                height_shift_range = 0.2,
                shear_range = 0.2,
                zoom_range = 0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory = self.config.training_data,
            subset = 'training',
            shuffle = True,
            **dataflow_kwargs
        )

    def save_model(self, path: Path, model: tf.keras.Model):
        model.save(path)
        
    def train(self, callback_list: list):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs = self.config.params_epochs,
            steps_per_epoch = self.steps_per_epoch,
            validation_steps = self.validation_steps,
            validation_data = self.valid_generator,
            callbacks = callback_list
        )

        self.save_model(
            path = self.config.trained_model_path,
            model = self.model
        )

In [10]:
from plantDiseaseDetection.components.prepare_callback import PrepareCallback
from plantDiseaseDetection.config import ConfigurationManager as CallbackConfigManager

In [11]:
try:
    config = ConfigurationManager()

    # Prepare Callbacks
    callback_config_manager = CallbackConfigManager()
    prepare_callbacks_config = callback_config_manager.get_prepare_callbacks_config()
    prepare_callback = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callback.get_tb_ckpt_callback()

    training_config = config.get_training_config()
    training = Traing(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(
        callback_list = callback_list
    )

except Exception as e:
    raise e

2023-11-14 09:06:35,239 : plantDiseaseDetection.logger.logger_config - INFO - YAML file: config/config.yaml loaded successfully
2023-11-14 09:06:35,243 : plantDiseaseDetection.logger.logger_config - INFO - YAML file: params.yaml loaded successfully
2023-11-14 09:06:35,244 : plantDiseaseDetection.logger.logger_config - INFO - created directory at: artifacts
2023-11-14 09:06:35,246 : plantDiseaseDetection.logger.logger_config - INFO - YAML file: config/config.yaml loaded successfully
2023-11-14 09:06:35,249 : plantDiseaseDetection.logger.logger_config - INFO - YAML file: params.yaml loaded successfully
2023-11-14 09:06:35,250 : plantDiseaseDetection.logger.logger_config - INFO - created directory at: artifacts
2023-11-14 09:06:35,251 : plantDiseaseDetection.logger.logger_config - INFO - created directory at: artifacts/prepare_callbacks/tensorboard_log_dir
2023-11-14 09:06:35,252 : plantDiseaseDetection.logger.logger_config - INFO - created directory at: artifacts/prepare_callbacks/checkp

Found 96 images belonging to 5 classes.
Found 387 images belonging to 5 classes.
Epoch 1/5
24/24 [==============================] - ETA: 0s - loss: 2.4262 - accuracy: 0.2264

/home/utpal108/dev/Python/PW_Skills_Projects/plant-disease-detection/venv/lib/python3.10/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


24/24 [==============================] - 36s 1s/step - loss: 2.4262 - accuracy: 0.2264 - val_loss: 1.6344 - val_accuracy: 0.2083
Epoch 2/5
24/24 [==============================] - 36s 1s/step - loss: 1.6364 - accuracy: 0.2183 - val_loss: 1.5821 - val_accuracy: 0.2708
Epoch 3/5
24/24 [==============================] - 35s 1s/step - loss: 1.6093 - accuracy: 0.2453 - val_loss: 1.5500 - val_accuracy: 0.3750
Epoch 4/5
24/24 [==============================] - 35s 1s/step - loss: 1.5945 - accuracy: 0.2426 - val_loss: 1.5732 - val_accuracy: 0.2604
Epoch 5/5
24/24 [==============================] - 36s 1s/step - loss: 1.5637 - accuracy: 0.2561 - val_loss: 1.5193 - val_accuracy: 0.3125
